### Wiener process and Geometric Brownian Motion (GBM)

In [2]:
from math import sqrt, exp
import numpy as np

In [3]:
def rnorm(mu=0, sigma=1):
    return np.random.normal(0, 1)


def wiener_process(T):
    wiener = np.zeros(T)
    for t in range(T):
        Wt = sqrt(t + 1) * rnorm(0, 1)
        wiener[t] = Wt
        
    return wiener


def GBM(S0, mu, sigma, T):
    gbm = np.zeros(T + 1)
    gbm[0] = S0
    wiener = wiener_process(T)
    for t in range(1, T + 1):
        drift = mu - (sigma ** 2) / 2
        drift = drift * t
        vol = sigma * wiener[t - 1]
        St = S0 * exp(drift + vol)
        gbm[t] = St
    
    return gbm

#### Geometric Brownian Motion (GBM) moments
$\mathrm{E}[S_t] = S_0 \cdot e^{\mu t}, \, t=1,\ldots, T$

$\mathrm{Var}[S_t] = S_0^2 \cdot e^{2\mu t} \cdot (e^{\sigma^2 t} - 1), \, t=1,\ldots, T$

In [4]:
def generate_sample(S0, mu, sigma, T, sample_size):
    sample = np.zeros(shape=(sample_size, T + 1))
    for i in range(sample_size): 
        gbm = GBM(S0, mu, sigma, T)
        sample[i] = gbm
        
    return sample


def gbm_mean(S0, mu, t):
    return S0 * exp(mu * t)


def gbm_std(S0, mu, sigma, t):
    std = S0 ** 2
    std *= exp(2 * mu * t)
    std *= (exp(sigma ** 2 * t) - 1)
    
    return sqrt(std)

##### Test case 1
For $\mu = 0$ we expect $\mathrm{E}[S_t] = S_0, \, t=1,\ldots, T$

In [5]:
sample_size = 100000
T = 10
S0 = 40
mu = 0
sigma = 0.25

sample = generate_sample(S0, mu, sigma, T, sample_size)
sample.mean(axis=0)

array([40.        , 40.02206942, 39.95538497, 40.10240357, 39.99762424,
       40.00142392, 39.99905924, 39.88780907, 39.99222446, 40.05903688,
       40.17748609])

##### Test case 2

For $\mu > 0$ we expect $\mathbb{E}[S_t] = S_0 \cdot e^{\mu t}, \, t=1,\ldots, T$

In [6]:
sample_size = 100000
S = 10
T = 10
S0 = 40
mu = 0.1
sigma = 0.25

sample = generate_sample(S0, mu, sigma, T, sample_size)
sample.mean(axis=0)

array([ 40.        ,  44.16307329,  48.92222195,  53.97934854,
        59.60157876,  65.84058505,  72.97872583,  81.06834048,
        89.27261241,  98.54901203, 109.17185374])

In [7]:
for t in range(0, T + 1):
    print(gbm_mean(S0, mu, t))

40.0
44.20683672302591
48.85611032640679
53.99435230304013
59.672987905650814
65.94885082800513
72.88475201562036
80.55010829881907
89.02163713969871
98.384124446278
108.7312731383618


### Shortest path algorithms

In [90]:
import networkx as nx
import itertools
import matplotlib.pyplot as plt

In [71]:
class ShortestPathResult():
    def __init__(self):
        self.dist = {}
        self.prev = {}

#### Dijkstra - basic implementation

In [78]:
def reconstruct_shortest_path(source, target, prev):
    if source != target:
        path = reconstruct_shortest_path(source, prev[target], prev)
        path.append(target)
        return path
    else:
        path = [source]
        return path

def dijkstra(G, source, target, weight='weight'):
    memo = ShortestPathResult()
    memo.prev = {node: None for node in G.nodes}
    memo.dist = {node: np.Inf for node in G.nodes}
    memo.dist[source] = 0
    memo.unvisited = memo.dist.copy()
    current = source
    while (len(memo.unvisited) > 0) & (current != target):
        for nbr, attrs in G[current].items():
            if not nbr in memo.unvisited:
                continue
            d = attrs[weight] 
            new_dist = memo.dist[current] + d
            if new_dist < memo.dist[nbr]:
                memo.dist[nbr] = new_dist
                memo.prev[nbr] = current
   
        del memo.unvisited[current]
        current = min(memo.unvisited, key=memo.unvisited.get)    
    shortest_path = reconstruct_shortest_path(source, target, memo.prev)
    
    return shortest_path, memo.dist[target]

#### Dynamic programming

##### Top-down approach

We will use the fact that $SP(s, t) = \min\{SP(s, u) + w(u, t) | (u, t) \in E\}$

First top-down approach is implemented. I.e., we start from target node and recourse until we end up at the source.

Requirements:
* Graph should be directed, i.e., need to support innode operation

In [65]:
def sp_dp_recurse(G, target, memo, weight='weight'):
    if target in memo.dist:
        return memo.dist[target]
    memo.dist[target] = np.Inf
    memo.prev[target] = None
    for node in G.predecessors(target):
        new_dist = sp_dp_recurse(G, node, memo) + G[node][target][weight]
        if new_dist < memo.dist[target]:
            memo.dist[target] = new_dist
            memo.prev[target] = node
            
    return memo.dist[target]
    

def shortest_path_dp(G, source, target, weight='weight'):
    memo = ShortestPathResult()
    memo.dist[source] = 0 # base state
    memo.prev[source] = None
    dist = sp_dp_recurse(G, target, memo)
    shortest_path = reconstruct_shortest_path(source, target, memo.prev)
    
    return shortest_path, memo.dist[target]

##### Bottom-up approach

We also implement bottom-up approach. I.e., we start at source at build solution until we reach the target.

This imposes an additional requirement:
* Graph should be [topologically sorted](https://en.wikipedia.org/wiki/Topological_sorting#:~:text=In%20computer%20science%2C%20a%20topological,before%20v%20in%20the%20ordering.) - linear ordering of its vertices such that for every directed edge uv from node u to node v, u comes before v in the ordering

Actually, due to the structure of our graph, topological sort is not needed. The easiest is to start from source and keep track in deque which node should be visited next.
We iterate until deque is empty or we reach target node.

In [92]:
from collections import deque

def shortest_path_dp_bottomup(G, source, target, weight='weight'):
#    G = nx.topological_sort(G)
    memo = ShortestPathResult()
    memo.prev = {node: None for node in G.nodes}
    memo.dist = {node: np.Inf for node in G.nodes}
    memo.dist[source] = 0
    current = source
    q = deque()
    q.append(source)
    while (len(q) > 0) & (current != target):
        for nbr in G.successors(current):
            q.append(nbr)
            new_dist = memo.dist[current] + G[current][nbr][weight]
            if new_dist < memo.dist[nbr]:
                memo.dist[nbr] = new_dist
                memo.prev[nbr] = current
        current = q.popleft()    
    shortest_path = reconstruct_shortest_path(source, target, memo.prev)
    
    return shortest_path, memo.dist[target]

#### Test case

In [ ]:
ss = list(itertools.product(range(T + 1), range(S + 1)))
nodes = [f'S_{stage}_{state}' for stage, state in ss]
edges = []
for stage, state in ss:
    if stage == T:
        continue
    for i in range(5):
        next_state = state + i
        if S < next_state:
            continue
        weight = np.random.normal(loc=10, scale=2)
        edge = (f'S_{stage}_{state}', f'S_{stage + 1}_{next_state}', {'weight':weight})
        edges.append(edge)
        
G = nx.DiGraph(edges)
"""
G = nx.DiGraph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
"""

In [97]:
path1 = nx.shortest_path(G, 'S_0_0', 'S_10_10', weight='weight')
path2 = dijkstra(G, 'S_0_0', 'S_10_10')[0]
path1 == path2

True

In [104]:
dist1 = dijkstra(G, 'S_0_0', 'S_10_10')[1]
dist2 = shortest_path_dp(G, 'S_0_0', 'S_10_10')[1]
dist3 = shortest_path_dp_bottomup(G, 'S_0_0', 'S_10_10')[1]
dist1 == dist2 == dist3

True

In [109]:
%%timeit
nx.shortest_path(G, 'S_0_0', 'S_10_10', weight='weight')

217 µs ± 3.04 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [110]:
%%timeit
shortest_path_dp(G, 'S_0_0', 'S_10_10')

476 µs ± 6.69 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [111]:
%%timeit
dijkstra(G, 'S_0_0', 'S_10_10')

977 µs ± 8.83 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [112]:
%%timeit
shortest_path_dp_bottomup(G, 'S_0_0', 'S_10_10')

810 ms ± 6.91 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


##### Conclusions

1. Networkx package has very well optimized shortest path algorithms (Dijkstra by default)
2. Dynamic programming top-down approach seems to efficient for small networks
3. Dynamic programming bottom-up (to me the most inuitive approach) is an order of magnitude slower than other algorithms 

### Buses electric fleet problem
**write down what we are trying to solve - DON'T FORGET**

In [155]:
def operating_cost(bias, slope, random_realization):
    return bias + slope * random_realization


def investment_cost(scale, random_realization1, random_realization2):
    return scale * random_realization1 * random_realization2


def generate_graph(T, S, investment_scale, investment_tech_obs, investment_rate_obs, diesel_bias, diesel_slope, diesel_obs,
                    electric_bias, electric_slope, electric_obs, yearly_max_buses, annual_discount):
    ss = list(itertools.product(range(T + 1), range(S + 1)))
    edges = []
    for stage, state in ss:
        if stage == T:
            investment = investment_cost(investment_scale, investment_tech_obs[stage], investment_rate_obs[stage])
            cost = (S - state) * investment * annual_discount[T]
            edge = (f'S_{stage}_{state}', f'S_final', {'weight':cost})
            edges.append(edge)
        else:
            for i in range(yearly_max_buses + 1):
                next_state = state + i
                if S < next_state:
                    continue
                investment = investment_cost(investment_scale, investment_tech_obs[stage], investment_rate_obs[stage])
                operating_diesel = operating_cost(diesel_bias, diesel_slope, diesel_obs[stage])
                operating_electric = operating_cost(electric_bias, electric_slope, electric_obs[stage])
                cost = (state + i) * operating_electric
                cost += (S - state - i) * operating_diesel
                cost += i * investment
                cost *= annual_discount[stage]
                edge = (f'S_{stage}_{state}', f'S_{stage + 1}_{next_state}', {'weight':cost})
                edges.append(edge)

    return nx.DiGraph(edges)


def descriptive_statistics(np_array):
    d = {}
    d['min'] = np.min(np_array)
    d['median'] = np.median(np_array)
    d['mean'] = np.mean(np_array)
    d['max'] = np.max(np_array)
    
    return d

#### Perfect Information

In [163]:
sample_size = 100000
T = 10
discount_factor = 0.98

random_diesel = generate_sample(40, 0, 0.25, T, sample_size)
random_electric = generate_sample(150, 0, 0.25, T, sample_size)
random_investment_tech = generate_sample(1, -0.05, 0.25, T, sample_size)
random_investment_rate = generate_sample(1, 0, 0.25, T, sample_size)
annual_discount = np.full(T + 1, discount_factor)
annual_discount = np.cumprod(annual_discount)

In [164]:
investment_scale = 4 * 10 ** 5
diesel_bias = 3600
diesel_slope = 155
electric_bias = 0
electric_slope = 115
yearly_max_buses = 4
paths = []
costs = []
for i in range(sample_size):
    G = generate_graph(T, S, investment_scale, random_investment_tech[i], random_investment_rate[i],
                       diesel_bias, diesel_slope, random_diesel[i], electric_bias, electric_slope,
                       random_electric[i], yearly_max_buses, annual_discount)
    
    path, cost = shortest_path_dp(G, 'S_0_0', 'S_final')
    paths.append(path)
    costs.append(cost)

In [165]:
costs = np.round(costs)

In [166]:
descriptive_statistics(costs)

{'min': 685540.0, 'median': 1587065.0, 'mean': 1633799.63803, 'max': 5062634.0}

#### Static Policy

In [172]:
expected_diesel = np.full(T + 1, 40)
expected_electric = np.full(T + 1, 150)
expected_investment_tech = np.zeros(T + 1)
expected_investment_rate = np.full(T + 1, 1)
for t in range(T + 1):
    expected_investment_tech[t] = np.exp(-0.05 * t)

G = generate_graph(T, S, investment_scale, expected_investment_tech, expected_investment_rate,
                       diesel_bias, diesel_slope, expected_diesel, electric_bias, electric_slope,
                       expected_electric, yearly_max_buses, annual_discount)
path, cost = shortest_path_dp(G, 'S_0_0', 'S_final')

In [174]:
round(cost)

2821089.0

In [175]:
path

['S_0_0',
 'S_1_0',
 'S_2_0',
 'S_3_0',
 'S_4_0',
 'S_5_0',
 'S_6_0',
 'S_7_0',
 'S_8_0',
 'S_9_0',
 'S_10_0',
 'S_final']

### Gurobi optimization

In [7]:
import gurobipy as gp
from gurobipy import GRB
import math

In [82]:
m = gp.Model('so')
at = [m.addVar(vtype=GRB.INTEGER, name=f'a_{t}') for t in range(T)]
st = [m.addVar(vtype=GRB.INTEGER, name=f's_{t}') for t in range(T + 1)]

m.update()

In [83]:
electric = [150 * 115 * (st[i] + at[i]) for i in range(T)]
diesel = [(3600 + 155 * 40) * (10 - st[i] - at[i]) for i in range(T)]
invest = [4*10**5 * math.exp(-0.05*t) * at[i] for i in range(T)]
final = [(10 - st[T]) * (4 * 10 ** 5) * math.exp(-0.05 * T)]

In [84]:
obj = []
obj.extend(electric)
obj.extend(diesel)
obj.extend(invest)
obj.extend(final)

m.setObjective(sum(obj))

In [26]:
try:

    # Create a new model
    m = gp.Model("mip1")

    # Create variables
    x = m.addVar(vtype=GRB.BINARY, name="x")
    y = m.addVar(vtype=GRB.BINARY, name="y")
    z = m.addVar(vtype=GRB.BINARY, name="z")

    # Set objective
    m.setObjective(x + y + 2 * z, GRB.MAXIMIZE)

    # Add constraint: x + 2 y + 3 z <= 4
    m.addConstr(x + 2 * y + 3 * z <= 4, "c0")

    # Add constraint: x + y >= 1
    m.addConstr(x + y >= 1, "c1")

    # Optimize model
    m.optimize()

    for v in m.getVars():
        print('%s %g' % (v.varName, v.x))

    print('Obj: %g' % m.objVal)

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0xf43f5bdf
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 16 available processors)

Solution count 2: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
x 1
y 0
z 1
Obj: 3
